In [ ]:
import numpy as np
from numpy.testing import assert_almost_equal

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage
from lsst.meas.mosaic.updateExposure import (applyMosaicResultsCatalog,
                                             applyMosaicResultsExposure, getWcs)

In [2]:
frame = 1
dispL1 = afwDisplay.getDisplay(frame)
dispL1.setMaskTransparency(75)
frame += 1
dispL2 = afwDisplay.getDisplay(frame)
dispL2.setMaskTransparency(75)
frame += 1
dispL3 = afwDisplay.getDisplay(frame)
dispL3.setMaskTransparency(75)
frame += 1
dispL4 = afwDisplay.getDisplay(frame)
dispL4.setMaskTransparency(75)

In [ ]:
rootDirL1 = "/tigress/HSC/HSC/rerun/lauren/LSST/DM-9862/cosmos_noJunk/4mnQuarterTest/"
rootDirL2 = "/tigress/HSC/HSC/rerun/lauren/LSST/DM-9862a/cosmos_noJunk/"

# make a butler
butlerL1 = dafPersist.Butler(rootDirL1)
butlerL2 = dafPersist.Butler(rootDirL2)

In [ ]:
visit = 1228
ccdList = [49, 50, 100, 101, 102, 103]
filter = "HSC-I"
tract = 0
srcIndex = 100

In [ ]:
for i, ccd in enumerate(ccdList):
    dataId = {"visit": visit, "ccd": ccd, "tract": tract}
    srcCatL2 = butlerL2.get("src", dataId)
    dataRef2 = butlerL2.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
    result2 = applyMosaicResultsCatalog(dataRef2, srcCatL2, True)
    cat2 = result2.catalog
    s = cat2[srcIndex]
    print ccd, float(s.getRa().asDegrees()), float(s.getDec().asDegrees()), s.getX(), s.getY(), s.getModelFlux()
    # assert_almost_equal(float(s.getRa().asDegrees()), knownRa[i], decimal=9)
    # assert_almost_equal(float(s.getDec().asDegrees()), knownDec[i], decimal=9)
    # assert_almost_equal(s.getX(), knownX[i], decimal=8)
    # assert_almost_equal(s.getY(), knownY[i], decimal=8)
    # assert_almost_equal(s.getModelFlux(), knownModelFlux[i], decimal=7)

In [5]:
knownRa = [150.291755265, 149.915585934, 150.761245879, 149.406088728, 150.77650489, 149.407912134]
knownDec = [2.20870255223, 2.16278669124, 1.84687842649, 1.84855672765, 2.55532853412, 2.55168410547]
knownX = [853.076542845, 218.0, 990.451759767, 109.935039499, 694.488613548, 1873.73688523]
knownY = [186.383389529, 210.0, 289.913434466, 256.140451952, 298.552189014, 275.660772235]
knownzModelFlux = [1477.6487901, 3948.61992604, 5022.95936829, 9588.90750291, 3467.05160473, 72949.3415395]

In [4]:
for i, ccd in enumerate(ccdList):
    dataId = {"visit": visit, "ccd": ccd, "tract": tract}
    srcCatL1 = butlerL1.get("src", dataId)
    dataRef1 = butlerL1.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
    result1 = applyMosaicResultsCatalog(dataRef1, srcCatL1, True)
    cat1 = result1.catalog
    s = cat1[srcIndex]
    print ccd, float(s.getRa().asDegrees()), float(s.getDec().asDegrees()), s.getX(), s.getY(), s.getModelFlux()

49 150.291755265 2.20870255223 853.076542845 186.383389529 1477.6487901
50 150.091436657 2.23827562198 218.0 210.0 3948.61992607
100 150.729960521 1.81164948552 990.451759767 289.913434466 4797.46466258
101 149.415087207 1.77390881637 109.935039499 256.140451952 9485.85455599
102 150.636950792 2.57691845954 694.488613548 298.552189014 3360.4136159
103 149.326371191 2.61841765656 1873.73688523 275.660772235 72026.6284466


In [5]:
ccd = 50
dataId = {"visit": visit, "ccd": ccd, "tract": tract}
dataRef1 = butlerL1.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
dataRef2 = butlerL2.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
expL1 = butlerL1.get("calexp", dataId)
expL2 = butlerL2.get("calexp", dataId)

In [28]:
dispL1.mtv(expL1.getMaskedImage(), title="LSST image new")
dispL2.mtv(expL2.getMaskedImage(), title="LSST image orig")

In [30]:
upExpL1 = applyMosaicResultsExposure(dataRef1, calexp=None)
upExpL1 = upExpL1.exposure
upExpL2 = applyMosaicResultsExposure(dataRef2, calexp=None)
upExpL2 = upExpL2.exposure

<class 'lsst.afw.image.imageLib.TanWcs'>

<class 'lsst.afw.image.imageLib.TanWcs'>



In [31]:
dispL3.mtv(upExpL1.getMaskedImage(), title="LSST image update new")
dispL4.mtv(upExpL2.getMaskedImage(), title="LSST image update orig")

In [5]:
srcCatL1 = butlerL1.get("src", dataId)
srcCatL2 = butlerL2.get("src", dataId)

In [6]:
s1 = srcCatL1[10]
s2 = srcCatL2[10]

In [7]:
s1.getCoord()

IcrsCoord(149.90675487730076*afwGeom.degrees, 2.2354969394502535*afwGeom.degrees)

In [8]:
s2.getCoord()

IcrsCoord(149.90675487730076*afwGeom.degrees, 2.2354969394502535*afwGeom.degrees)

In [9]:
dataRef1 = butlerL1.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
result1 = applyMosaicResultsCatalog(dataRef1, srcCatL1, True)
dataRef2 = butlerL2.dataRef("src", dataId=dict(visit=visit, ccd=ccd, tract=tract))
result2 = applyMosaicResultsCatalog(dataRef2, srcCatL2, True)

HERE!!...
1: ffp.wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
2: ffp.wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]
2: wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
HERE!!...
1: ffp.wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]
2: ffp.wcs.getCDMatrix() =  [[ -7.79102403e-10   4.68748816e-05]
 [  4.68612723e-05   8.23129204e-09]]
2: wcs.getCDMatrix() =  [[  7.79102403e-10  -4.68748816e-05]
 [ -4.68612723e-05  -8.23129204e-09]]


In [10]:
cat1 = result1.catalog
cat2 = result2.catalog

In [11]:
cat1[10].getCoord()

IcrsCoord(149.9068054393456*afwGeom.degrees, 2.2354459115086627*afwGeom.degrees)

In [12]:
cat2[10].getCoord()

IcrsCoord(150.10030301657136*afwGeom.degrees, 2.165451941318979*afwGeom.degrees)

In [29]:
dispL1.mtv(expL1.getMaskedImage(), title="LSST image")

In [30]:
dispL2.mtv(expL2.getMaskedImage(), title="LSST image")

In [15]:
wcs = getWcs(dataRef1)

In [16]:
wcs.getCDMatrix()

array([[ -7.79102403e-10,   4.68748816e-05],
       [  4.68612723e-05,   8.23129204e-09]])